In [1]:
%run mathutil.ipynb
np.random.seed(1234)       #난수발생패턴 고정
def randomize(): np.random.seed(time.time())

In [2]:
class Model(object):
    #객체 초기화
    def __init__(self, name, dataset):
        self.name = name
        self.dataset = dataset
        self.is_training = False
        if not hasattr(self, 'rand_std'): self.rand_std = 0.030
    
    #출력문으로 객체를 출력할 때의 출력 문자열 생성 방법을 정의한다.
    def __str__(self):
        return '{}/{}'.format(self.name, self.dataset)

    #전체 과정을 실행시키는 메인 함수 역할(학습, 평가, 시각화 메서드를 차례로 호출)
    def exec_all(self, epoch_count=10, batch_size=10, learning_rate=0.001,
                 report=0, show_cnt=3):
        self.train(epoch_count, batch_size, learning_rate, report)
        self.test()
        if show_cnt > 0: self.visualize(show_cnt) #시각화 메서드는 시각화 출력 대상 데이터 수가 0보다 클 떄만 호출

In [3]:
class MlpModel(Model):
    def __init__(self, name, dataset, hconfigs):
        #기반 클래스인 Model 클래스를 찾아 그 객체 초기화 함수를 호출하여 name, dataset값을 저장한다.
        super(MlpModel, self).__init__(name, dataset)
        #신경망이 이용할 파라미터를 준비
        self.init_parameters(hconfigs)

In [4]:
def mlp_init_parameters(self, hconfigs):
    self.hconfigs = hconfigs
    self.pm_hiddens = []

    #입출력 벡터 크기는 전역변수가 아니라 dataset 객체의 속성값으로부터 얻는다
    prev_shape = self.dataset.input_shape

    for hconfig in hconfigs:
        pm_hidden, prev_shape = self.alloc_layer_param(prev_shape, hconfig)
        #생성된 파라미터들을 전역 변수에 저장하는 대신 객체 변수로 저장한다.
        self.pm_hiddens.append(pm_hidden)

    output_cnt = int(np.prod(self.dataset.output_shape))
    self.pm_output, _ = self.alloc_layer_param(prev_shape, output_cnt)
        
def mlp_alloc_layer_param(self, input_shape, hconfig):
    input_cnt = np.prod(input_shape)
    output_cnt = hconfig

    weight, bias = self.alloc_param_pair([input_cnt, output_cnt])

    return {'w':weight, 'b':bias}, output_cnt

def mlp_alloc_param_pair(self, shape):
    weight = np.random.normal(0, self.rand_std, shape)
    bias = np.zeros([shape[-1]])
    return weight, bias

#이 코드에서 정의된 함수들을 클래스의 멤버 함수, 즉 메서드로 등록한다.
#이때 정의된 함수 이름과 등록되는 메서드 이름이 서로 다른데 이처럼 두 이름을 독립적으로 부여하면 파생 클래스에서
#메서드를 재정의할때 함수 이름을 달리 할 수 있어서 프로그램의 가독성이 높아짐
MlpModel.init_parameters = mlp_init_parameters
MlpModel.alloc_layer_param = mlp_alloc_layer_param
MlpModel.alloc_param_pair = mlp_alloc_param_pair

In [5]:
def mlp_model_train(self, epoch_count=10, batch_size=10, \
                    learning_rate=0.001, report=0):
    #학습률 파라미터를 객체 변수를 이용해 전달
    self.learning_rate = learning_rate
    
    batch_count = int(self.dataset.train_count / batch_size)
    time1 = time2 = int(time.time())
    if report != 0:
        print('Model {} train started:'.format(self.name))

    for epoch in range(epoch_count):
        costs = []
        accs = []
        #데이터 뒤섞기 기능을 따로 독립시켜 호출
        self.dataset.shuffle_train_data(batch_size*batch_count)
        for n in range(batch_count):
            trX, trY = self.dataset.get_train_data(batch_size, n)
            cost, acc = self.train_step(trX, trY)
            costs.append(cost)
            accs.append(acc)

        if report > 0 and (epoch+1) % report == 0:
            #get_validate_data()메서드를 호출해 그때그때 새로 받아오는 제한된 수의 검증용 데이터를 이용한다.
            vaX, vaY = self.dataset.get_validate_data(100)
            acc = self.eval_accuracy(vaX, vaY)
            time3 = int(time.time())
            tm1, tm2 = time3-time2, time3-time1
            #확장에 대비해 검증 단계 출력을 데이터셋의 train_prt_result() 메서드에 의뢰해 생성한다.
            #실행 시간 추적 정보를 전달하여 출력에 반영할 수 있게 했다.
            self.dataset.train_prt_result(epoch+1, costs, accs, acc, tm1, tm2)
            time2 = time3

    tm_total = int(time.time()) - time1
    print('Model {} train ended in {} secs:'.format(self.name, tm_total))
    
MlpModel.train = mlp_model_train

In [6]:
#exec_all()메서드가 평가를 위해 호출하는 메서드
def mlp_model_test(self):
    teX, teY = self.dataset.get_test_data()
    time1 = int(time.time())
    acc = self.eval_accuracy(teX, teY)
    time2 = int(time.time())
    self.dataset.test_prt_result(self.name, acc, time2-time1)

MlpModel.test = mlp_model_test

In [7]:
#시각화 메서드 정의
def mlp_model_visualize(self, num):
    print('Model {} Visualization'.format(self.name))
    #데이터셋에서 시각화용 데이터를 얻은 후
    deX, deY = self.dataset.get_visualize_data(num)
    #get_estimate()메서드를 실행시켜 신경망 추정 정보를 얻는다.
    est = self.get_estimate(deX)
    #신경망 추정 정보를 데이터셋의 시각화 메서드에 제공해 시각화 출력을 생성하며 이때 원래의 입력 및 출력 정보도 함께 제공
    self.dataset.visualize(deX, est, deY)

MlpModel.visualize = mlp_model_visualize

In [8]:
#미니배치 학습 메서드 정의
#is_training 플래그의 제어 기능이 추가된 점만 다름, 이 플래그는 train()메서드가 아니라 이 메서드에서 제어해야함

def mlp_train_step(self, x, y):
    self.is_training = True
    
    output, aux_nn = self.forward_neuralnet(x)
    loss, aux_pp = self.forward_postproc(output, y)
    accuracy = self.eval_accuracy(x, y, output)
    
    G_loss = 1.0
    G_output = self.backprop_postproc(G_loss, aux_pp)
    self.backprop_neuralnet(G_output, aux_nn)

    self.is_training = False

    return loss, accuracy

MlpModel.train_step = mlp_train_step

In [9]:
#신경망 부분에 대한 순전파 메서드 정의
def mlp_forward_neuralnet(self, x):
    hidden = x
    aux_layers = []

    for n, hconfig in enumerate(self.hconfigs):
        hidden, aux = self.forward_layer(hidden, hconfig, self.pm_hiddens[n])
        aux_layers.append(aux)

    #출력 계층은 hconfig 인숫값 대신 None을 전달하는 방법으로 출력 계층임을 나타내도록 함
    output, aux_out = self.forward_layer(hidden, None, self.pm_output)
    
    #리턴값을 아래 역전파 메서드에 전달
    return output, [aux_out, aux_layers]

#신경망 부분에 대한 역전파 메서드 정의
def mlp_backprop_neuralnet(self, G_output, aux):
    aux_out, aux_layers = aux
    
    G_hidden = self.backprop_layer(G_output, None, self.pm_output, aux_out)
    
    for n in reversed(range(len(self.hconfigs))):
        hconfig, pm, aux = self.hconfigs[n], self.pm_hiddens[n], aux_layers[n]
        G_hidden = self.backprop_layer(G_hidden, hconfig, pm, aux)
        
    return G_hidden

MlpModel.forward_neuralnet = mlp_forward_neuralnet
MlpModel.backprop_neuralnet = mlp_backprop_neuralnet

In [10]:
#계층 하나에 대한 순전파 메서드 정의
def mlp_forward_layer(self, x, hconfig, pm):
    #선형 연산
    y = np.matmul(x, pm['w']) + pm['b']
    #비선형 활성화 함수 관련처리 -> 은닉 계층에서만 적용(은닉 계층인지 출력 계층인지는 hconfig 인숫값이 None인지를 검사)
    if hconfig is not None: y = relu(y)
    return y, [x,y]

#계층 하나에 대한 역전파 메서드 정의
def mlp_backprop_layer(self, G_y, hconfig, pm, aux):
    x, y = aux
    #비선형 활성화 함수 관련처리 -> 은닉 계층에서만 적용(은닉 계층인지 출력 계층인지는 hconfig 인숫값이 None인지를 검사)
    if hconfig is not None: G_y = relu_derv(y) * G_y
    
    #선형 연산에 대한 역전파 처리
    g_y_weight = x.transpose()
    g_y_input = pm['w'].transpose()
    
    G_weight = np.matmul(g_y_weight, G_y)
    G_bias = np.sum(G_y, axis=0)
    G_input = np.matmul(G_y, g_y_input)
    
    pm['w'] -= self.learning_rate * G_weight
    pm['b'] -= self.learning_rate * G_bias

    return G_input

MlpModel.forward_layer = mlp_forward_layer
MlpModel.backprop_layer = mlp_backprop_layer

In [11]:
#후처리 부분에 대한 순전파 메서드 정의
def mlp_forward_postproc(self, output, y):
    #데이터셋 객체의 메서드를 호출해 손실 함숫값을 실제로 계산하도록 요청
    #이때, 역전파용 보조 정보를 aux_loss에 챙겨두었다가 역전파때 제공
    loss, aux_loss = self.dataset.forward_postproc(output, y)
    #forward_extra_cost()메서드를 호출해 추가적인 별도의 손실 성분 loss_extra와 역전파용 보조 정보 aux_extra를 보고한다.
    extra, aux_extra = self.forward_extra_cost(y)
    return loss + extra, [aux_loss, aux_extra]

def mlp_forward_extra_cost(self, y):
    #0,None이라는 의미 없는 값은 아직 아무런 정규화 장치도 도입되지 않아 계산에 반영할 추가 손실 성분이 없기 때문
    return 0, None

MlpModel.forward_postproc = mlp_forward_postproc
MlpModel.forward_extra_cost = mlp_forward_extra_cost

In [12]:
#후처리 부분에 대한 역전파 메서드 정의
def mlp_backprop_postproc(self, G_loss, aux):
    aux_loss, aux_extra = aux
    #순전파때 호출한 forward_extra_cost()메서드와 짝을 이루는 메서드로서 정규화 기법을 도입할 때를 대비해 준비되었지만
    #당장은 아무러 할일이 없으므로 아래 정의된 pass로 빈 함수 형태로 정의
    self.backprop_extra_cost(G_loss, aux_extra)
    #loss의 손실 기울기 G_loss로부터 신경망 출력 output의 손실 기울기 G_output을 계산
    G_output = self.dataset.backprop_postproc(G_loss, aux_loss)
    return G_output

def mlp_backprop_extra_cost(self, G_loss, aux):
    pass

MlpModel.backprop_postproc = mlp_backprop_postproc
MlpModel.backprop_extra_cost = mlp_backprop_extra_cost

In [13]:
#정확도 계산 메서드 정의
def mlp_eval_accuracy(self, x, y, output=None):
    if output is None:
        #이를 검사하여 값 지정이 누락된 경우 forward_neuralnet()메서드를 호출해 직접 순전파 처리를 수행
        output, _ = self.forward_neuralnet(x)
    #추정의 정확도를 계산하는 구체적인 과정은 데이터셋 객체의 메서드에 맡겨 처리
    accuracy = self.dataset.eval_accuracy(x, y, output)
    return accuracy

MlpModel.eval_accuracy = mlp_eval_accuracy

In [14]:
#추정 결과 산출 메서드 정의
#신경망이 로짓값이나 로짓값 벡터 등으로 구해낸 결과를 x값으로 전달받아 확률값이나 확률 분포 등 출력 유형에 따른
#의미 있는 값으로 바꾸어 estimate 변수에 담아 보고하는 것이 이 함수의 역할이다.
def mlp_get_estimate(self, x):
    output, _ = self.forward_neuralnet(x)
    #시각화에 적합한 형태의 추정 결과를 얻어내는 구체적인 과정은 데이터셋 객체의 메서드를 호출해 처리
    estimate = self.dataset.get_estimate(output)
    return estimate

MlpModel.get_estimate = mlp_get_estimate